In [1]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_ibm_runtime.fake_provider import FakeAlmadenV2
from qiskit.circuit.library import TwoLocal


import numpy as np
import pandas as pd
from qiskit_algorithms.utils import algorithm_globals
import networkx as nx
from networkx.algorithms import bipartite

import matplotlib.pyplot as plt

import random

from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, Binary, maximize
from scipy.optimize import minimize
from itertools import combinations

import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit.visualization import *
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.primitives import Estimator
from qiskit.primitives import Sampler,  BackendSampler, BackendSamplerV2

from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Estimator
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.circuit.library import QAOAAnsatz

from qiskit.circuit.library import EfficientSU2

from qiskit_algorithms.optimizers import SPSA

from qiskit_algorithms import SamplingVQE
from qiskit.primitives import Sampler
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA


In [10]:
import numpy as np
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_aer import AerSimulator
from qiskit.primitives import BackendSampler
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA

# Number of donors and recipients
n_donors = 5
n_recipients = 5

# Weights for donor-recipient pairs (example values)
weights = [
    [10, 2, 1, 6, 8],   # Donor 0
    [5, 9, 4, 9, 11],   # Donor 1
    [7, 3, 2, 1, 8],    # Donor 2
    [6, 8, 1, 3, 5],    # Donor 3
    [1, 2, 5, 6, 7]     # Donor 4
]

# Define the number of binary variables
n = n_donors + n_recipients

# Initialize the Q matrix
Q = np.zeros((n, n))

# Objective terms (maximize suitability)
for i in range(n_donors):
    for j in range(n_recipients):
        Q[i, n_donors + j] = -weights[i][j]  # Note the negative sign for maximization

# Penalty term to enforce one-to-one matching
penalty = 10  # Adjust penalty strength as needed

# Penalties for donors to connect with only one recipient
for i in range(n_donors):
    for j1 in range(n_recipients):
        for j2 in range(j1 + 1, n_recipients):
            Q[i, n_donors + j1] += penalty
            Q[i, n_donors + j2] += penalty

# Penalties for recipients to connect with only one donor
for j in range(n_recipients):
    for i1 in range(n_donors):
        for i2 in range(i1 + 1, n_donors):
            Q[i1, i2] += penalty

# Convert Q matrix into QUBO
qubo = QuadraticProgram()
for i in range(n):
    qubo.binary_var(name=f'x_{i}')
for i in range(n):
    for j in range(n):
        if Q[i, j] != 0:
            qubo.objective.set_quadratic_coefficients(i, j, Q[i, j])

# Print Q matrix for debugging
print("QUBO Matrix (Q):")
print(Q)

# Setup for QAOA
backend = AerSimulator(max_parallel_threads=1)
sampler = BackendSampler(backend=backend)
optimizer = COBYLA()
qaoa = QAOA(sampler=sampler, optimizer=optimizer, reps=4)

# Use the QAOA with the minimum eigen optimizer
minimum_eigen_optimizer = MinimumEigenOptimizer(qaoa)

print("Solving the problem...")
result = minimum_eigen_optimizer.solve(qubo)

# Extract and print results
print("Solution:", result)
print("Objective value:", result.fval)

# Extract and print matched pairs
matched_pairs = []
for i in range(n_donors):
    for j in range(n_recipients):
        index = i * n_recipients + j
        if result.x[index] > 0.5:
            matched_pairs.append((i, j))
            print(f"Donor {i} matched with Recipient {j}")


TypeError: 'dok_matrix' object is not callable